In [ ]:
import sklearn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from PyRadioLoc.Utils.GeoUtils import GeoUtils

In [ ]:
def get_distance_array(y_pred, y_test):
    dist = list()
    for y_sample, y_pred in  zip(y_test, y_pred):
        dist.append(GeoUtils.distanceInKm(y_sample[0], y_sample[1], y_pred[0], y_pred[1]))
    return dist

def get_quality_metrics(y_pred, y_test):
    dist = get_distance_array(y_pred, y_test)
    localization_error_min = min(dist)*1000
    localization_error_max = max(dist)*1000
    localization_error_mean = np.mean(dist)*1000
    localization_error_standard_deviation = np.std(dist)*1000
    print("min:            %0.2f m" % localization_error_min)
    print("max:            %0.2f m" % localization_error_max)
    print("mean:           %0.2f m" % localization_error_mean)
    print("std deviation:  %0.2f m" % localization_error_standard_deviation)
    
def write_to_csv(y_pred_lat, y_pred_lon, test_id, file_path):
    #y_pred_dataframe = pd.DataFrame(y_pred, columns=['lat', 'lon'])
    y_pred_dataframe = pd.concat([y_pred_lat, y_pred_lon, test_id], axis=1)
    y_pred_dataframe.to_csv(path_or_buf=file_path, index = False)
    
def y_RSSIs(file):
    y = pd.read_csv(file)
    y_rssi_1_1 = y.drop(["rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_1_2 = y.drop(["rssi_1_1","rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_1_3 = y.drop(["rssi_1_1","rssi_1_2", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_2_1 = y.drop(["rssi_1_1","rssi_1_2", "rssi_1_3", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_2_2 = y.drop(["rssi_1_1","rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_3", "rssi_3_1", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_2_3 = y.drop(["rssi_1_1","rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_3_1", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_3_1 = y.drop(["rssi_1_1","rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_2", "rssi_3_3"], axis = 1)
    y_rssi_3_2 = y.drop(["rssi_1_1","rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_3"], axis = 1)
    y_rssi_3_3 = y.drop(["rssi_1_1","rssi_1_2", "rssi_1_3", "rssi_2_1", "rssi_2_2", "rssi_2_3", "rssi_3_1", "rssi_3_2"], axis = 1)
    
    return y_rssi_1_1, y_rssi_1_2, y_rssi_1_3, y_rssi_2_1, y_rssi_2_2, y_rssi_2_3, y_rssi_3_1, y_rssi_3_2, y_rssi_3_3

def Results(y_test, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    results = [mean_absolute_error(y_test, y_pred), mean_squared_error(y_test, y_pred), np.sqrt(mean_squared_error(y_test, y_pred)), r2_score(y_test, y_pred)]
    return results

def ResultsMeters(y_test, y_pred):
    y_test_np = y_test.to_numpy()
    #y_test_np
    y_pred_np = y_pred.to_numpy()
    #y_pred_np
    return get_quality_metrics(y_test_np, y_pred_np)  

# Implementação LightGBM:

In [ ]:
import lightgbm as lgb

def LightGBM_TCC(X_train, y_train, X_test):

    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    d_train = lgb.Dataset(X_train, label=y_train)
    #d_train
    params = {}
    params['learning_rate'] = 1.0
    params['boosting_type'] = 'gbdt'#'gbdt'
    params['metric'] = 'mae'
    params['sub_feature'] = 0.8
    params['num_leaves'] = 255
    params['min_data'] = 1
    #params['max_depth'] = 10
    clf = lgb.train(params, d_train, 10000)
    #Prediction
    y_pred=clf.predict(X_test)
    #y_pred     
    
    return y_pred

def y_pred_write_File(X_train, y_train_rssi_1_1, y_train_rssi_1_2, y_train_rssi_1_3, y_train_rssi_2_1, y_train_rssi_2_2, y_train_rssi_2_3, y_train_rssi_3_1, y_train_rssi_3_2, y_train_rssi_3_3, X_test, Metodo_Num):
    #y_train_lat, y_train_lon#
     
    y_pred_rssi_1_1 = LightGBM_TCC(X_train, y_train_rssi_1_1, X_test)
    y_pred_rssi_1_2 = LightGBM_TCC(X_train, y_train_rssi_1_2, X_test)
    y_pred_rssi_1_3 = LightGBM_TCC(X_train, y_train_rssi_1_3, X_test)
    
    y_pred_rssi_2_1 = LightGBM_TCC(X_train, y_train_rssi_2_1, X_test)
    y_pred_rssi_2_2 = LightGBM_TCC(X_train, y_train_rssi_2_2, X_test)
    y_pred_rssi_2_3 = LightGBM_TCC(X_train, y_train_rssi_2_3, X_test)
    
    y_pred_rssi_3_1 = LightGBM_TCC(X_train, y_train_rssi_3_1, X_test)
    y_pred_rssi_3_2 = LightGBM_TCC(X_train, y_train_rssi_3_2, X_test)
    y_pred_rssi_3_3 = LightGBM_TCC(X_train, y_train_rssi_3_3, X_test)
    
    #y_pred_lat = LightGBM_TCC(X_train, y_train_lat, X_test)
    #y_pred_lon = LightGBM_TCC(X_train, y_train_lon, X_test)
    
    #y_pred_lat = pd.DataFrame({'lat': y_pred_lat})
    #y_pred_lat.head()
    #y_pred_lon = pd.DataFrame({'lon': y_pred_lon})
    #y_pred_lon.head()
    
    y_pred_rssi_1_1 = pd.DataFrame({'rssi_1_1':  y_pred_rssi_1_1})
    y_pred_rssi_1_2 = pd.DataFrame({'rssi_1_2':  y_pred_rssi_1_2})
    y_pred_rssi_1_3 = pd.DataFrame({'rssi_1_3':  y_pred_rssi_1_3})
    
    y_pred_rssi_2_1 = pd.DataFrame({'rssi_2_1':  y_pred_rssi_2_1})
    y_pred_rssi_2_2 = pd.DataFrame({'rssi_2_2':  y_pred_rssi_2_2})
    y_pred_rssi_2_3 = pd.DataFrame({'rssi_2_3':  y_pred_rssi_2_3})
    
    y_pred_rssi_3_1 = pd.DataFrame({'rssi_3_1':  y_pred_rssi_3_1})
    y_pred_rssi_3_2 = pd.DataFrame({'rssi_3_2':  y_pred_rssi_3_2})
    y_pred_rssi_3_3 = pd.DataFrame({'rssi_3_3':  y_pred_rssi_3_3})
    
    y_pred = pd.concat([y_pred_rssi_1_1, y_pred_rssi_1_2, y_pred_rssi_1_3, y_pred_rssi_2_1, y_pred_rssi_2_2, y_pred_rssi_2_3, y_pred_rssi_3_1, y_pred_rssi_3_2, y_pred_rssi_3_3], axis=1)
    #y_pred.head()
    #write_to_csv(y_pred, "ResultTCC/Resultados_Metodo_{}.csv".format(Metodo_Num))
    
    return y_pred

In [ ]:
X_train = pd.read_csv("databasesTCC/Outdoor_X_train_80%.csv")
X_train = X_train.drop("lat", axis = 1)
X_train = X_train.drop("lon", axis = 1)
X_train = X_train.drop("idx", axis = 1)
X_train.head()

# Targets de Treinamento: Arquivo Outdoor_y_train_80%

In [ ]:
#y_train lat and Lon Outdoor 80%
#y_train_lat, y_train_lon 
y_train_rssi_1_1, y_train_rssi_1_2, y_train_rssi_1_3, y_train_rssi_2_1, y_train_rssi_2_2, y_train_rssi_2_3, y_train_rssi_3_1, y_train_rssi_3_2, y_train_rssi_3_3 = y_RSSIs("DatabasesTCC/Outdoor_y_train_80%.csv")
y_train_rssi_1_1.head()